In [76]:
import sys
reload(sys)
sys.setdefaultencoding('utf8')

In [1]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.pipeline import Pipeline

In [2]:
%run query_features.py
%run scripts/helper.py
%run scripts/model_train_plus_test.py

In [3]:
crowd_train = load_file('./data/train.csv/train.csv', None)
crowd_test = load_file('./data/test.csv/test.csv', None)

In [4]:
# crowd_train = crowd_train[crowd_train.relevance_variance < 0.5]

In [5]:
target = crowd_train.median_relevance.values

In [6]:
# train_index, test_index = ssSplit(target, train_size=8000, random_state=44)

In [7]:
# Xt = crowd_train.iloc[train_index]
# Xv = crowd_train.iloc[test_index]

Xt = crowd_train
Xv = crowd_test

In [8]:
# yt = target[train_index]
# yv = target[test_index]

yt = target

In [9]:
# correct_map = build_query_correction_map(Xt, crowd_test)

In [10]:
# def spell_correct_query(x):
#     if x not in correct_map:
#         return x
#     else:
#         return correct_map[x]
    
# Xt['query'] = Xt['query'].map(spell_correct_query)

In [11]:
# Xv['query'] = Xv['query'].map(spell_correct_query)

In [ ]:
Xt_tweaked = tweak_text(Xt)
Xv_tweaked = tweak_text(Xv)

In [237]:
Xfitted, tfv = TFIDF(Xt_tweaked, None)

In [238]:
svd = TruncatedSVD(n_components=200, algorithm='randomized', n_iter=5, random_state=None, tol=0.0)

scl = StandardScaler(copy=True, with_mean=True, with_std=True)

clf = SVC(C=10.0, kernel='rbf', degree=3, 
        gamma=0.0, coef0=0.0, shrinking=True, probability=False, 
        tol=0.001, cache_size=200, class_weight=None, 
        verbose=False, max_iter=-1, random_state=None)

keywords = keyword_counter(Xt)

In [239]:
features = stack([keywords, Xfitted])

In [240]:
pipeline = Pipeline([('svd', svd), ('scl', scl), ('clf', clf)])

In [241]:
pipeline.fit(features, yt)

MemoryError: 

In [188]:
keywords_test = keyword_counter(Xv)

In [189]:
Xtest = tfv.transform(Xv_tweaked)

In [190]:
features_test = stack([keywords_test, Xtest])

In [191]:
preds_new_model = pipeline.predict(features_test)

In [192]:
# make_submission(crowd_test.id.values.astype(int), preds_new_model, 'spell_correct_preds_rel_var.csv')

In [193]:
print 'Kappa score on validation set ', (quadratic_weighted_kappa(yv, preds_new_model))

Kappa score on validation set  0.582931573728


## Best score

In [128]:
best_score_df = pd.read_csv('./submissions/addtitional_features_500.csv')

In [129]:
best_score = best_score_df.prediction

In [130]:
ensemble = (preds_new_model + best_score) / 2
ensemble_int = [int(score) for score in ensemble]

In [131]:
make_submission(crowd_test.id.values.astype(int), ensemble_int, 'spell_correct_rel_var_600.csv')